In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import os

In [2]:
from pathlib import Path

BASE_DIR = Path.cwd().resolve().parents[1]

#print("Project root:", BASE_DIR)

In [3]:
directory=BASE_DIR / "data"/ "APS" / "raw"

In [4]:
for filename in os.listdir(directory):
    
    plant=filename[:-5]
    print(plant)

African Daisy
African Violet
Alfalfa
Almond
Anemone
Apple
Apricot
Asparagus
Avocado
Azalea
Banana and Plantain
Barley
Bean
Bedding Plants
Beet
Bellflower
Black Walnut
Bleeding Heart
Blueberry
Butterfly Flower
Cacao
Caneberries
Carnation
Carrot
Cassava
Cattleya Lindl. spp
Celery
Chickpea
Chrysanthemum
Cineraria
Citrus
Coconut Palm
Coffee
Common House or Florist's Hydrangea
Conifer
Corn (syn. Maize)
Cotton
Cranberry
Crucifers
Cucurbits
Cultivated Wild Rice
Cyclamen
Date Palm
Douglas-fir
Elm
English (Persian) Walnut
European Hazelnut
Flax
Foliage Plants (House Plants)
Fuchsia
Garden Dahlia
Geranium
Grape
Hemp
Holiday Cacti
Hop
Impatiens
Jerusalem Cherry
Kalanchoë
Lentil
Lettuce
Lingonberry
Lisianthus
Mango
Mimulus, Monkey-Flower
Mint
Mustard
Oats
Onion and Garlic
Papaya
Passionfruit
Pea
Peach and Nectarine
Peanut
Pear
Pearl Millet
Pecan
Pepper
Persian Violet
Pigeonpea
Pineapple
Pistachio
Pocketbook Plant
Poinsettia
Potato
Primulas
Rapeseed = Canola
Red Clover
Rhododendron
Rice
Rose
Rye
Sa

In [5]:
data = []
current_category = None
current_disease = None
current_pathogens = []

for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    with open(filepath, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    plant=filename[:-5]

    soup = BeautifulSoup(html_content, 'html.parser')
    
    for element in soup.find_all(['h3', 'dt', 'dd']):
        if element.name == 'h3':
            # Если есть накопленные данные о предыдущем заболевании, сохраняем их
            if current_disease and current_pathogens:
                data.append({
                    'Category': current_category,
                    'Disease': current_disease,
                    'Pathogen': '\n'.join(current_pathogens),
                    'Afflict': plant
                })
                current_pathogens = []
            
            current_category = element.get_text(strip=True)
            current_disease = None
            
        elif element.name == 'dt':
            # Если есть накопленные данные о предыдущем заболевании, сохраняем их
            if current_disease and current_pathogens:
                data.append({
                    'Category': current_category,
                    'Disease': current_disease,
                    'Pathogen': '\n'.join(current_pathogens),
                    'Afflict': plant
                })
                current_pathogens = []
            
            current_disease = element.get_text(strip=True)
            
        elif element.name == 'dd' and current_disease:
            pathogen_text = element.get_text(strip=True)
            current_pathogens.append(pathogen_text)
    
    # Добавляем последнее заболевание
    if current_disease and current_pathogens:
        data.append({
            'Category': current_category,
            'Disease': current_disease,
            'Pathogen': '\n'.join(current_pathogens),
            'Afflict': plant
        })

df = pd.DataFrame(data)

In [6]:
df

,Category,Disease,Pathogen,Afflict
0,BACTERIAL DISEASES,Bacterial leaf spot,Pseudomonas cichorii(Swingle 1925) Stapp 1928,African Daisy
1,BACTERIAL DISEASES,Southern wilt,Ralstonia solanacearum(Smith 1896) Yabuuchi et...,African Daisy
2,FUNGAL DISEASES,Ascochyta leaf spot,Ascochyta doroniciAllesch.,African Daisy
3,FUNGAL DISEASES,Alternaria leaf spot,Alternaria alternata(Fr.:Fr.) Keissl.\nA. dauc...,African Daisy
4,FUNGAL DISEASES,Black root rot,Thielaviopsis basicola(Berk. & Broome) Ferrari...,African Daisy
...,...,...,...,...
5564,VIRAL AND VIROID DISEASES,Wheat yellow leaf,GenusClosterovirus; Wheat yellow leaf virus(WYLV),Wheat
5565,VIRAL AND VIROID DISEASES,Wheat yellow mosaic,GenusBymovirus; Wheat yellow mosaic virus(WYMV...,Wheat
5566,VIRAL AND VIROID DISEASES,Winter wheat mosaic,GenusTenuivirus; Winter wheat mosaic virus(WWMV)d,Wheat
5567,VIRAL AND VIROID DISEASES,Winter wheat (Russian) mosaic,GenusNucleorhabdovirus; Winter wheat Russian m...,Wheat


In [7]:
df.to_csv(BASE_DIR / "data"/ "APS" / 'plant_diseases.csv', index=False)

In [8]:
df.iloc[4]['Pathogen']

'Thielaviopsis basicola(Berk. & Broome) Ferraris\nChalara elegansNag Raj & Kendr. [synanamorph]'